## Build UniRef50 clusters
We want to build the train, valid, and test splits using the uniref50 clusters.
This will give us a better understanding of the performance of the model since proteins that were left out from a particular cluster should be more difficult to recover


In [1]:
# first download the mappings from uniprot IDs to uniref50 (and a lot of others)
# see here for more details: https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/idmapping/README
!wget -O /projects/deepgreen/jlaw/inputs/uniprot/2021-05/2021-06-07-idmapping_selected.tab.gz \
  ftp://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/idmapping/idmapping_selected.tab.gz

--2021-06-07 10:46:00--  ftp://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/idmapping/idmapping_selected.tab.gz
           => ‘/projects/deepgreen/jlaw/inputs/uniprot/2021-05/2021-06-07-idmapping_selected.tab.gz’
Resolving ftp.uniprot.org (ftp.uniprot.org)... 128.175.240.195
Connecting to ftp.uniprot.org (ftp.uniprot.org)|128.175.240.195|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/databases/uniprot/current_release/knowledgebase/idmapping ... done.
==> SIZE idmapping_selected.tab.gz ... 10401229504
==> PASV ... done.    ==> RETR idmapping_selected.tab.gz ... done.
Length: 10401229504 (9.7G) (unauthoritative)

100%[===================================>] 10,401,229,504 57.9MB/s   in 3m 2s  

2021-06-07 10:49:07 (54.5 MB/s) - ‘/projects/deepgreen/jlaw/inputs/uniprot/2021-05/2021-06-07-idmapping_selected.tab.gz’ saved [10401229504]



In [12]:
idmapping_file = "/projects/deepgreen/jlaw/inputs/uniprot/2021-05/2021-06-07-idmapping_selected.tab.gz"
column_names = ['UniProtKB-AC', 'UniProtKB-ID', 'GeneID (EntrezGene)', 'RefSeq', 'GI', 'PDB', 'GO', 'UniRef100', 'UniRef90', 'UniRef50', 'UniParc', 'PIR', 'NCBI-taxon', 'MIM', 'UniGene', 'PubMed', 'EMBL', 'EMBL-CDS', 'Ensembl', 'Ensembl_TRS', 'Ensembl_PRO', 'Additional PubMed']

In [14]:
!gzip -cd $idmapping_file | head -n 3

Q6GZX4	001R_FRG3G	2947773	YP_031579.1	81941549; 49237298		GO:0046782	UniRef100_Q6GZX4	UniRef90_Q6GZX4	UniRef50_Q6GZX4	UPI00003B0FD4		654924			15165820	AY548484	AAT09660.1				
Q6GZX3	002L_FRG3G	2947774	YP_031580.1	49237299; 81941548		GO:0033644; GO:0016021	UniRef100_Q6GZX3	UniRef90_Q6GZX3	UniRef50_Q6GZX3	UPI00003B0FD5		654924			15165820	AY548484	AAT09661.1				
Q197F8	002R_IIV3	4156251	YP_654574.1	109287880; 123808694; 106073503			UniRef100_Q197F8	UniRef90_Q197F8	UniRef50_Q197F8	UPI0000D83464		345201			16912294	DQ643392	ABF82032.1				

gzip: stdout: Broken pipe


In [19]:
uniref_clusters_file = os.path.dirname(idmapping_file) + "/uniref-clusters.tab.gz"

In [17]:
# extract just the uniref cluster ids
# !gzip -cd $idmapping_file | cut -f 1,8,9,10 | head
!gzip -cd $idmapping_file | cut -f 1,8,9,10 | gzip > $uniref_clusters_file

In [ ]:
import pandas as pd
import os, sys
import gzip
from tqdm import tqdm
from collections import defaultdict

In [20]:
names = ['uniprot-id', 'uniref100-id', 'uniref90-id', 'uniref50-id']
df = pd.read_csv(uniref_clusters_file, sep='\t', names=names)
df.head()

,uniprot-id,uniref100-id,uniref90-id,uniref50-id
0,Q6GZX4,UniRef100_Q6GZX4,UniRef90_Q6GZX4,UniRef50_Q6GZX4
1,Q6GZX3,UniRef100_Q6GZX3,UniRef90_Q6GZX3,UniRef50_Q6GZX3
2,Q197F8,UniRef100_Q197F8,UniRef90_Q197F8,UniRef50_Q197F8
3,Q197F7,UniRef100_Q197F7,UniRef90_Q197F7,UniRef50_Q197F7
4,Q6GZX2,UniRef100_Q6GZX2,UniRef90_Q6GZX2,UniRef50_Q6GZX2


In [25]:
# limit the uniprot IDs to those that have a GO annotation with an EXPC evidence code
expc_prots_file = "/projects/deepgreen/jlaw/inputs/goa/2021-05/prots-with-expc-ann.txt"
expc_prots = pd.read_csv(expc_prots_file, header=None, squeeze=True)
expc_prots

0         A0A009IHW8
1         A0A021WW32
2         A0A021WZA4
3         A0A023FBW4
4         A0A023FBW7
             ...    
138115        X6RJY0
138116        X6RKS3
138117        X6RLN4
138118        X6RLP6
138119        X6RLR1
Name: 0, Length: 138120, dtype: object

In [27]:
expc_prots = set(expc_prots.tolist())

In [28]:
df_expc = df[df['uniprot-id'].isin(expc_prots)]

In [31]:
# now write it 
out_file = "/projects/deepgreen/jlaw/inputs/uniprot/2021-05/for-goa/uniref-clusters-expc.tab.gz"
df_expc.to_csv(out_file, sep='\t', index=None)

In [34]:
df_expc['uniref50-id'].value_counts()

UniRef50_P01246        53
UniRef50_A0A0B4K823    43
UniRef50_Q07163        40
UniRef50_P04578        32
UniRef50_Q9V3H7        29
                       ..
UniRef50_P38892         1
UniRef50_P26969         1
UniRef50_Q62304         1
UniRef50_Q52LA3         1
UniRef50_P9WGR4         1
Name: uniref50-id, Length: 93696, dtype: int64

I'm surprised the uniref50 clusters are relactively small. The largest cluster has only 53 members

In [35]:
df_expc['uniref90-id'].value_counts()

UniRef90_P01246        35
UniRef90_Q07163        30
UniRef90_Q6V6S2        26
UniRef90_A0A0B4K869    20
UniRef90_Q9VZW3        20
                       ..
UniRef90_W7K228         1
UniRef90_A0A6A9KFQ6     1
UniRef90_Q92903         1
UniRef90_P17038         1
UniRef90_A0A1D8PSV5     1
Name: uniref90-id, Length: 112175, dtype: int64

In [36]:
df_expc['uniref100-id'].value_counts()

UniRef100_Q67HN6        17
UniRef100_P0DP23        17
UniRef100_Q6ENF9        16
UniRef100_A0A0E0KU15    13
UniRef100_Q969X1        13
                        ..
UniRef100_Q3URK3         1
UniRef100_P70859         1
UniRef100_F2XNG5         1
UniRef100_Q9URZ1         1
UniRef100_Q46507         1
Name: uniref100-id, Length: 130420, dtype: int64